In [ ]:
!pip install crewai==0.28.8 crewai-tools langchain-groq langchain-community langchain_core
!pip install pandas requests python-dotenv wikipedia openpyxl # Added openpyxl for reading excel

In [ ]:
!pip install pyngrok
from flask import Flask, request, jsonify
from pyngrok import ngrok

In [ ]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from crewai import Agent, Task, Crew
from crewai_tools import tool
from langchain_community.retrievers import WikipediaRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
import re

# ====================
# ✅ Load Environment Variables
# ====================
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

# ====================
# ✅ Wikipedia Chatbot Setup
# ====================
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

# Arabic and English retrievers
retriever_ar = WikipediaRetriever(top_k_results=2, lang="ar")
retriever_en = WikipediaRetriever(top_k_results=2, lang="en")

# LLM setup for Wikipedia chatbot
wikipedia_llm = ChatGroq(model="meta-llama/llama-4-maverick-17b-128e-instruct")

# Wikipedia prompt
wikipedia_prompt = ChatPromptTemplate.from_template("""
You are a helpful travel assistant specializing in Egypt. Answer the question based on the context provided.
If the context doesn't contain relevant information, use your general knowledge about Egypt to provide a helpful answer.
Keep your responses informative but concise.
Context: {context}
Question: {question}
""")

# Format docs function
def format_docs(docs):
    if not docs:
        return "No specific information found."
    return "\n\n".join(doc.page_content for doc in docs)

# Combined retriever function
def combined_retriever_fn(question):
    try:
        docs_ar = retriever_ar.invoke(question)
        docs_en = retriever_en.invoke(question)
        return docs_ar + docs_en
    except Exception as e:
        print(f"Wikipedia retrieval error: {e}")
        return []

# Wrap the combined retriever and formatter using RunnableLambda
combined_retriever = RunnableLambda(combined_retriever_fn)

def format_context_and_question(inputs):
    if isinstance(inputs, str):
        # If inputs is just a string (the question), create the proper structure
        context = combined_retriever_fn(inputs)
        return {
            "context": format_docs(context),
            "question": inputs
        }
    else:
        # If inputs is already a dict
        return {
            "context": format_docs(inputs.get("context", [])),
            "question": inputs.get("question", "")
        }

format_step = RunnableLambda(format_context_and_question)

# Wikipedia chain - Fixed to handle string input properly
wikipedia_chain = (
    RunnableLambda(lambda x: x)  # Pass through the question
    | format_step
    | wikipedia_prompt
    | wikipedia_llm
    | StrOutputParser()
)

# ====================
# ✅ Load Data Files (Travel Planner)
# ====================
def load_data():
    """Load all data files once and return them"""
    try:
        print("Loading travel data files...")
        data = {
            'hotels': pd.read_csv("Egypt_hotels_data.csv") if os.path.exists("Egypt_hotels_data.csv") else pd.DataFrame(),
            'attractions': pd.read_csv("FINAL _Atrreaction dataset.csv") if os.path.exists("FINAL _Atrreaction dataset.csv") else pd.DataFrame(),
            'events': pd.read_csv("Egypt_Expanded_Events_No_Website_Full_Details23.csv") if os.path.exists("Egypt_Expanded_Events_No_Website_Full_Details23.csv") else pd.DataFrame(),
            'restaurants': pd.read_csv("Egypt_Restaurants.csv") if os.path.exists("Egypt_Restaurants.csv") else pd.DataFrame(),
            'transportation': pd.read_csv("Egypt_Transportation_Realistic_Expanded.csv") if os.path.exists("Egypt_Transportation_Realistic_Expanded.csv") else pd.DataFrame(),
            'flights': pd.read_csv("flights_data_egypt_with_realistic_prices.csv") if os.path.exists("flights_data_egypt_with_realistic_prices.csv") else pd.DataFrame(),
            'medical_tourism': pd.read_excel("medical_tourism_prices_egp_converted.xlsx") if os.path.exists("medical_tourism_prices_egp_converted.xlsx") else pd.DataFrame()
        }

        # Optimize data types and clean up
        for name, df in data.items():
            if not df.empty:
                if 'City' in df.columns:
                    df['City'] = df['City'].str.strip().str.lower()
                if 'city' in df.columns:
                    df['city'] = df['city'].str.strip().str.lower()
                print(f"✅ Loaded {name}: {len(df)} records")
            else:
                print(f"⚠️ {name}: No data found")

        return data
    except Exception as e:
        print(f"❌ Data loading error: {e}")
        return {}

# Load data once
DATA = load_data()

# ====================
# ✅ City Detection Function
# ====================
def extract_city_from_text(user_input: str) -> str:
    """Extract Egyptian city name from user input"""
    # List of Egyptian cities (both English and common variations)
    egyptian_cities = {
        'cairo': ['cairo', 'قاهرة', 'giza', 'الجيزة'],
        'alexandria': ['alexandria', 'alex', 'الإسكندرية', 'اسكندرية'],
        'luxor': ['luxor', 'الأقصر', 'اقصر'],
        'aswan': ['aswan', 'أسوان', 'اسوان'],
        'hurghada': ['hurghada', 'الغردقة', 'غردقة'],
        'sharm el sheikh': ['sharm', 'sharm el sheikh', 'شرم الشيخ', 'شرم'],
        'dahab': ['dahab', 'دهب'],
        'marsa alam': ['marsa alam', 'مرسى علم'],
        'fayoum': ['fayoum', 'faiyum', 'الفيوم', 'فيوم'],
        'siwa': ['siwa', 'سيوة'],
        'gouna': ['gouna', 'el gouna', 'الجونة'],
        'nuweiba': ['nuweiba', 'نويبع'],
        'ain el sokhna': ['ain el sokhna', 'العين السخنة', 'sokhna'],
        'marsa matrouh': ['marsa matrouh', 'matrouh', 'مرسى مطروح', 'مطروح']
    }

    user_input_lower = user_input.lower()

    # Check for city mentions
    for city, variations in egyptian_cities.items():
        for variation in variations:
            if variation in user_input_lower:
                return city.title()

    return None

# ====================
# ✅ Question Classification System (Updated)
# ====================
def classify_user_intent(user_input: str) -> tuple:
    """
    Classify if user wants trip planning or general information
    Returns: (intent, city_name)
    """
    user_input_lower = user_input.lower()

    # Check for exit commands
    exit_commands = ['exit', 'quit', 'bye', 'goodbye', 'stop', 'end']
    if any(cmd in user_input_lower for cmd in exit_commands):
        return "exit", None

    # First check if a city is mentioned
    detected_city = extract_city_from_text(user_input)

    # Keywords that indicate trip planning
    planning_keywords = [
        'plan', 'trip', 'travel', 'visit', 'itinerary', 'recommend', 'suggest',
        'where to go', 'best city', 'vacation', 'holiday', 'tour', 'journey',
        'accommodation', 'hotel', 'stay', 'budget', 'days', 'week'
    ]

    # Keywords that indicate general questions
    general_keywords = [
        'what is', 'who is', 'when did', 'how did', 'why did', 'history of',
        'tell me about', 'explain', 'describe', 'information about', 'facts about',
        'culture', 'tradition', 'religion', 'politics', 'economy', 'geography'
    ]

    planning_score = sum(1 for keyword in planning_keywords if keyword in user_input_lower)
    general_score = sum(1 for keyword in general_keywords if keyword in user_input_lower)

    # If city is detected and planning keywords are present, it's trip planning
    if detected_city and planning_score > 0:
        return "trip_planning", detected_city
    # If city is detected but no clear planning keywords, ask for clarification
    elif detected_city:
        return "city_specific", detected_city
    # If planning keywords without city, use recommendation system
    elif planning_score > general_score:
        return "trip_planning", None
    else:
        return "general_question", None

# ====================
# ✅ Get Top 4 City Recommendations (Unchanged)
# ====================
def get_top_city_recommendations(user_profile: dict) -> list:
    """Get top 4 city recommendations based on user profile"""
    try:
        import calendar
        from datetime import datetime
        import re

        attractions_df = DATA.get('attractions')
        if attractions_df is None or attractions_df.empty:
            return [("Cairo", 100, "Historic capital with pyramids"),
                   ("Luxor", 90, "Ancient temples and tombs"),
                   ("Alexandria", 80, "Mediterranean coastal city"),
                   ("Aswan", 70, "Nubian culture and Nile scenery")]

        attractions_df['City'] = attractions_df['City'].astype(str).str.strip().str.lower()

        def extract_months_from_range(date_range_str):
            """Extract months from user date input"""
            date_range_str = str(date_range_str).strip().lower()
            month_list = [m.lower() for m in calendar.month_name if m]
            month_abbrev = [month[:3] for month in month_list]

            found_months = []
            for i, month in enumerate(month_list):
                if month in date_range_str or month_abbrev[i] in date_range_str:
                    found_months.append(month)

            if found_months:
                return found_months

            try:
                parts = re.split(r"\s*to\s*", date_range_str)
                if len(parts) == 2:
                    start_date = datetime.strptime(parts[0].strip(), "%d-%m-%Y")
                    end_date = datetime.strptime(parts[1].strip(), "%d-%m-%Y")
                    months = list({start_date.month, end_date.month})
                else:
                    single_date = datetime.strptime(parts[0].strip(), "%d-%m-%Y")
                    months = [single_date.month]
                return [calendar.month_name[m].lower() for m in months]
            except Exception:
                pass

            return []

        # Extract user preferences
        profile_months = extract_months_from_range(user_profile.get("dates", ""))
        profile_interests = [x.strip().lower() for x in user_profile.get("interests", "").split(",")]
        previous = [x.strip().lower() for x in user_profile.get("previous_places", "").split(",") if x.strip()]

        try:
            budget_value = float(''.join(filter(str.isdigit, user_profile.get("budget", "0"))))
            is_budget_low = budget_value < 5000
        except:
            is_budget_low = False

        def score_city(city_data, city_name):
            score = 0
            reasons = []

            # Interest matching
            if "Type" in city_data.columns:
                types_str = ' '.join(city_data["Type"].dropna().astype(str).str.lower())
                match_count = sum(1 for interest in profile_interests if interest in types_str)
                score += match_count * 10
                if match_count > 0:
                    reasons.append(f"Matches your interests: {', '.join(profile_interests[:2])}")

                if any("history" in interest or "ancient" in interest for interest in profile_interests):
                    if any(x in types_str for x in ["history", "ancient", "temple", "pyramid"]):
                        score += 15
                        reasons.append("Rich historical attractions")

            # Rating bonus
            if "Rating" in city_data.columns:
                ratings = pd.to_numeric(city_data["Rating"], errors='coerce').dropna()
                if not ratings.empty:
                    avg_rating = ratings.mean()
                    if avg_rating >= 4.5:
                        score += 12
                        reasons.append("Highly rated attractions")
                    elif avg_rating >= 4.0:
                        score += 7
                        reasons.append("Well-rated attractions")
                    elif avg_rating >= 3.5:
                        score += 3

            # Budget considerations
            if is_budget_low:
                budget_friendly_cities = ['luxor', 'aswan', 'fayoum', 'siwa']
                if city_name.lower() in budget_friendly_cities:
                    score += 8
                    reasons.append("Budget-friendly destination")

            # Penalty for previously visited places
            if city_name.lower() in previous:
                score -= 25
                reasons.append("Previously visited")

            # City-specific bonuses based on trip type
            trip_type = user_profile.get('trip_type', '').lower()
            if 'family' in trip_type:
                family_friendly = ['cairo', 'alexandria', 'hurghada', 'sharm el sheikh']
                if city_name.lower() in family_friendly:
                    score += 5
                    reasons.append("Family-friendly destination")
            elif 'solo' in trip_type:
                solo_friendly = ['cairo', 'luxor', 'dahab', 'siwa']
                if city_name.lower() in solo_friendly:
                    score += 5
                    reasons.append("Great for solo travelers")

            return score, '; '.join(reasons[:3])

        # Valid Egyptian cities
        valid_cities = [c.lower().strip() for c in [
            'Cairo', 'Alexandria', 'Luxor', 'Aswan', 'Fayoum', 'Dahab', 'Hurghada',
            'Marsa Alam', 'Gouna', 'Nuweiba', 'Sharm El Sheikh', 'Siwa',
            'Ain El Sokhna', 'Marsa Matrouh'
        ]]
        cities = [c for c in attractions_df["City"].dropna().unique() if c.strip().lower() in valid_cities]

        city_scores = {}
        for city in cities:
            city_data = attractions_df[attractions_df["City"] == city]
            score, reasons = score_city(city_data, city)
            if score > 0:
                city_scores[city.title()] = (score, reasons)

        if not city_scores:
            return [("Cairo", 100, "Historic capital with pyramids"),
                   ("Luxor", 90, "Ancient temples and tombs"),
                   ("Alexandria", 80, "Mediterranean coastal city"),
                   ("Aswan", 70, "Nubian culture and Nile scenery")]

        # Sort by score and return top 4
        top_cities = sorted(city_scores.items(), key=lambda x: x[1][0], reverse=True)[:4]
        return [(city, score, reasons) for city, (score, reasons) in top_cities]

    except Exception as e:
        print(f"Error getting recommendations: {e}")
        return [("Cairo", 100, "Historic capital with pyramids"),
               ("Luxor", 90, "Ancient temples and tombs"),
               ("Alexandria", 80, "Mediterranean coastal city"),
               ("Aswan", 70, "Nubian culture and Nile scenery")]

# ====================
# ✅ Travel Planner Tools (Multi-Agent System)
# ====================
@tool("City Analysis Tool")
def analyze_city(tool_input: str) -> str:
    """Analyze city data including hotels, attractions, restaurants, events."""
    try:
        if not tool_input or not isinstance(tool_input, str):
            return "❌ Please provide a valid city name for analysis"

        city = tool_input.strip().lower()

        # Load datasets
        hotels = DATA.get('hotels', pd.DataFrame())
        attractions = DATA.get('attractions', pd.DataFrame())
        restaurants = DATA.get('restaurants', pd.DataFrame())
        events = DATA.get('events', pd.DataFrame())

        # Filter per dataset
        city_hotels = hotels[hotels['City'].str.lower() == city] if not hotels.empty and 'City' in hotels.columns else pd.DataFrame()
        city_attractions = attractions[attractions['City'].str.lower() == city] if not attractions.empty and 'City' in attractions.columns else pd.DataFrame()
        city_restaurants = restaurants[restaurants['City'].str.lower() == city] if not restaurants.empty and 'City' in restaurants.columns else pd.DataFrame()
        city_events = events[events['City'].str.lower() == city] if not events.empty and 'City' in events.columns else pd.DataFrame()

        response = f"\n📍 **City Analysis: {city.title()}**\n\n"

        # Hotels
        if not city_hotels.empty:
            response += f"🏨 **Accommodation ({len(city_hotels)} options)**\n"
            if 'Rating' in city_hotels.columns:
                top_hotels = city_hotels.nlargest(3, 'Rating')
                response += "⭐ Top Rated Hotels:\n"
                for _, row in top_hotels.iterrows():
                    hotel_name = row.get('Hotel Name', row.get('Name', 'Hotel'))
                    rating = row.get('Rating', 'N/A')
                    response += f"  • {hotel_name} ({rating}/5)\n"
        else:
            response += "🏨 **Accommodation:** Limited data available\n"

        # Attractions
        if not city_attractions.empty:
            response += f"\n🏛 **Attractions ({len(city_attractions)} sites)**\n"
            if 'Rating' in city_attractions.columns:
                top_attr = city_attractions.nlargest(5, 'Rating')
                response += "⭐ Must-Visit Attractions:\n"
                for _, row in top_attr.iterrows():
                    attraction_name = row.get('Attraction', row.get('Name', 'Attraction'))
                    rating = row.get('Rating', 'N/A')
                    response += f"  • {attraction_name} ({rating}/5)\n"
        else:
            response += "\n🏛 **Attractions:** Limited data available\n"

        # Restaurants
        if not city_restaurants.empty:
            response += f"\n🍽 **Dining ({len(city_restaurants)} options)**\n"
            rating_col = 'User Rating' if 'User Rating' in city_restaurants.columns else 'Rating'
            if rating_col in city_restaurants.columns:
                top_restaurants = city_restaurants.nlargest(3, rating_col)
                response += "⭐ Top Rated Restaurants:\n"
                for _, row in top_restaurants.iterrows():
                    restaurant_name = row.get('Name', row.get(' Name', 'Restaurant'))
                    rating = row.get(rating_col, 'N/A')
                    response += f"  • {restaurant_name} ({rating}/5)\n"
        else:
            response += "\n🍽 **Dining:** Limited data available\n"

        return response

    except Exception as e:
        return f"❌ Analysis error: {str(e)}"

@tool("Weather Information Tool")
def get_weather(tool_input: str) -> str:
    """Get current weather information for a city."""
    try:
        if not tool_input or not isinstance(tool_input, str):
            return "❌ Please provide a valid city name for weather information"

        city = tool_input.strip()
        url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units=metric"
        res = requests.get(url, timeout=5)
        if res.status_code == 200:
            data = res.json()
            desc = data['weather'][0]['description']
            temp = data['main']['temp']
            feels_like = data['main']['feels_like']
            humidity = data['main']['humidity']
            return f"🌤 **Current Weather in {city.title()}:**\n- Condition: {desc.title()}\n- Temperature: {temp}°C (feels like {feels_like}°C)\n- Humidity: {humidity}%"
        else:
            return f"🌤 Weather data unavailable for {city}."
    except Exception as e:
        return f"🌤 Weather service temporarily unavailable."

@tool("Itinerary Creation Tool")
def create_itinerary(tool_input: str) -> str:
    """Create detailed daily itinerary for a city."""
    try:
        if not tool_input or not isinstance(tool_input, str):
            return "❌ Please provide city name"

        city = tool_input.strip()

        # Try to get duration from user profile, default to 3 days
        try:
            duration = int(user_profile.get('duration', 3))
        except:
            duration = 3

        attractions = DATA.get('attractions', pd.DataFrame())

        plan = f"🗺 **{duration}-Day Itinerary for {city.title()}**\n\n"

        if not attractions.empty and 'City' in attractions.columns:
            city_attractions = attractions[attractions['City'] == city.lower()]

            if not city_attractions.empty:
                # Get top attractions based on rating
                rating_col = 'Rating' if 'Rating' in city_attractions.columns else None
                if rating_col:
                    top_attractions = city_attractions.nlargest(duration * 2, rating_col)
                else:
                    top_attractions = city_attractions.head(duration * 2)

                for day in range(duration):
                    plan += f"### Day {day + 1}\n"
                    start_idx = day * 2
                    end_idx = min((day + 1) * 2, len(top_attractions))
                    day_attractions = top_attractions.iloc[start_idx:end_idx]

                    times = ["🌅 Morning", "☀️ Afternoon"]
                    for i, (_, attraction) in enumerate(day_attractions.iterrows()):
                        if i < len(times):
                            time_slot = times[i]
                            attraction_name = attraction.get('Attraction', f'Attraction {i+1}')
                            rating = attraction.get('Rating', 'N/A')
                            plan += f"- {time_slot}: {attraction_name}"
                            if rating != 'N/A':
                                plan += f" (⭐ {rating}/5)"
                            plan += "\n"
                    plan += "\n"
            else:
                plan += create_generic_itinerary(city, duration)
        else:
            plan += create_generic_itinerary(city, duration)

        return plan

    except Exception as e:
        return f"❌ Itinerary error: {str(e)}"

def create_generic_itinerary(city: str, duration: int) -> str:
    """Create a generic itinerary when no specific data is available"""
    generic_plan = ""
    city_lower = city.lower()

    if city_lower == "cairo":
        activities = [
            "Visit the Great Pyramids of Giza", "Explore Egyptian Museum",
            "Tour Islamic Cairo and Khan el-Khalili Bazaar", "Visit Coptic Cairo",
            "Explore Citadel of Saladin", "Stroll along the Nile Corniche"
        ]
    elif city_lower == "luxor":
        activities = [
            "Explore Valley of the Kings", "Visit Karnak Temple Complex",
            "Tour Luxor Temple", "Visit Hatshepsut Temple",
            "Take a Nile River cruise", "Explore local markets"
        ]
    elif city_lower == "alexandria":
        activities = [
            "Visit Bibliotheca Alexandrina", "Explore Qaitbay Citadel",
            "Walk along the Corniche", "Visit Catacombs of Kom el Shoqafa",
            "Explore Montaza Palace", "Try fresh seafood at local restaurants"
        ]
    elif city_lower == "aswan":
        activities = [
            "Visit Philae Temple", "Take a felucca ride on the Nile",
            "Explore Nubian Village", "Visit Unfinished Obelisk",
            "See Abu Simbel temples", "Enjoy sunset at Elephantine Island"
        ]
    else:
        activities = [
            "Explore the main attractions", "Visit local markets and shops",
            "Try traditional Egyptian cuisine", "Take a walking tour of historic areas",
            "Visit museums and cultural sites", "Enjoy local entertainment"
        ]

    for day in range(duration):
        generic_plan += f"### Day {day + 1}\n"
        day_activities = activities[day*2:(day+1)*2] if len(activities) > day*2 else activities[-2:]
        times = ["🌅 Morning", "☀️ Afternoon"]

        for i, activity in enumerate(day_activities):
            if i < len(times):
                generic_plan += f"- {times[i]}: {activity}\n"
        generic_plan += "\n"

    return generic_plan

# ====================
# ✅ Multi-Agent System Setup
# ====================
def setup_travel_agents():
    """Setup the multi-agent system for trip planning"""
    llm = ChatGroq(
        model="meta-llama/llama-4-maverick-17b-128e-instruct",
        api_key=GROQ_API_KEY,
        temperature=0.3,
        max_tokens=2000
    )

    city_analyzer = Agent(
        role="City Information Analyst",
        goal="Provide comprehensive city analysis including accommodations, attractions, dining, and weather",
        backstory="You are a local Egyptian travel expert with detailed knowledge of hotels, restaurants, attractions, and current conditions.",
        tools=[analyze_city, get_weather],
        llm=llm,
        verbose=False,
        allow_delegation=False
    )

    trip_planner = Agent(
        role="Travel Itinerary Designer",
        goal="Create detailed, practical day-by-day travel plans",
        backstory="You are a professional travel planner specializing in Egyptian destinations.",
        tools=[create_itinerary],
        llm=llm,
        verbose=False,
        allow_delegation=False
    )

    return city_analyzer, trip_planner

# ====================
# ✅ User Profile Collection (Updated)
# ====================
def collect_user_data(specific_city=None) -> dict:
    """Collect user preferences for trip planning"""
    if specific_city:
        print(f"\n📝 Great! I'll help you plan your trip to {specific_city}!")
        print("Please answer a few questions to personalize your itinerary:\n")
    else:
        print("\n📝 Let's personalize your Egypt trip! Please answer the following questions:\n")

    return {
        "trip_type": input("1. What type of traveler are you? (Solo/Family/Friends): ") or "Solo",
        "budget": input("2. What is your budget? (in EGP or USD): ") or "5000",
        "duration": input("3. How many days?: ") or "3",
        "dates": input("4. When are you traveling? (dates/months): ") or "flexible",
        "interests": input("5. Your main interests? (History/Beach/Adventure/Food): ") or "History,Culture",
        "previous_places": input("6. Places you've visited in Egypt before: ") or "None",
        "accommodation": input("7. Accommodation preference? (Hotel/Hostel/Airbnb): ") or "Hotel",
        "transportation": input("8. Transportation preference?: ") or "Car",
        "food": input("9. Dietary restrictions?: ") or "None",
        "activity_level": input("10. Activity level? (Few/Moderate/Full): ") or "Moderate",
        "sleep_schedule": input("11. Early riser or night explorer?: ") or "Flexible",
        "luxury_level": input("12. Comfort level? (Economy/Mid-range/Luxury): ") or "Mid-range",
        "selected_city": specific_city  # Add the specific city to the profile
    }

# ====================
# ✅ City Selection Process (Updated)
# ====================
def select_city_from_recommendations(user_profile: dict) -> str:
    """Let user select from top 4 recommended cities"""
    print("\n🎯 Based on your preferences, here are the top 4 recommended cities:\n")

    recommendations = get_top_city_recommendations(user_profile)

    for i, (city, score, reasons) in enumerate(recommendations, 1):
        print(f"{i}. 🏛️ **{city}** (Match Score: {score}%)")
        print(f"   📋 Why it's recommended: {reasons}")
        print()

    while True:
        try:
            choice = input("👉 Which city would you like to plan your trip for? (Enter 1-4): ").strip()
            choice_num = int(choice)

            if 1 <= choice_num <= 4:
                selected_city = recommendations[choice_num - 1][0]
                print(f"\n✅ Great choice! Planning your trip to {selected_city}...")
                return selected_city
            else:
                print("❌ Please enter a number between 1 and 4.")
        except ValueError:
            print("❌ Please enter a valid number (1-4).")

# ====================
# ✅ Updated Trip Planner
# ====================
def run_trip_planner_for_city(user_profile: dict, selected_city: str):
    """Run the multi-agent trip planning system for selected city"""
    try:
        city_analyzer, trip_planner = setup_travel_agents()

        task1 = Task(
            description=f"Analyze the city '{selected_city}' and provide comprehensive information about accommodations, attractions, restaurants, and current weather.",
            agent=city_analyzer,
            expected_output="Detailed city analysis with specific recommendations"
        )

        task2 = Task(
            description=f"Create a detailed {user_profile['duration']}-day itinerary for {selected_city} considering user preferences: {user_profile}.",
            agent=trip_planner,
            expected_output="Day-by-day itinerary with activities and logistics"
        )

        crew = Crew(
            agents=[city_analyzer, trip_planner],
            tasks=[task1, task2],
            verbose=False,
            process="sequential"
        )

        result = crew.kickoff()
        return result

    except Exception as e:
        return f"❌ Trip planning error: {str(e)}"


# ====================
# ✅ Main Entry Point
# ====================
def main():
    print("🌍 Welcome to the Egypt Travel Assistant!\n")

    user_input = input("🗣️ How can I help you today?\n> ")

    intent, detected_city = classify_user_intent(user_input)

    if intent == "general_question":
        print("\n🔎 Let me look that up for you...\n")
        response = wikipedia_chain.invoke(user_input)
        print(response)

    elif intent == "trip_planning":
        profile = collect_user_data(specific_city=detected_city)
        city_to_plan = detected_city or select_city_from_recommendations(profile)
        profile["selected_city"] = city_to_plan
        result = run_trip_planner_for_city(profile, city_to_plan)
        print("\n📋 Your Trip Plan:\n")
        print(result)

    elif intent == "city_specific":
        print(f"\n🧭 You asked about {detected_city}. Here's what I found:\n")
        city_summary = analyze_city.invoke(detected_city)
        weather = get_weather.invoke(detected_city)
        print(city_summary)
        print(weather)

        ask_plan = input("\n📌 Would you like me to create an itinerary for this city? (yes/no): ").strip().lower()
        if ask_plan in ['yes', 'y']:
            profile = collect_user_data(specific_city=detected_city)
            result = run_trip_planner_for_city(profile, detected_city)
            print("\n📋 Your Trip Plan:\n")
            print(result)
        else:
            print("\n👍 Got it! Let me know if you need anything else.")

    else:
        print("\n❓ Sorry, I didn't quite understand your request. Could you rephrase it?")


app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    user_input = data.get('message', '').strip()
    session = data.get('session', {})

    print("🔥 Incoming session:", session)
    print("🔥 User message:", user_input)

    if not user_input:
        return jsonify({"error": "No message provided", "session": session}), 400

    # ✅ Profile collection phase
    if session.get("awaiting_profile"):
        profile_fields = [
            "trip_type", "budget", "duration", "dates", "interests", "previous_places",
            "accommodation", "transportation", "food", "activity_level",
            "sleep_schedule", "luxury_level"
        ]
        profile_questions = [
            "1. What type of traveler are you? (Solo/Family/Friends):",
            "2. What is your budget? (in EGP or USD):",
            "3. How many days?:",
            "4. When are you traveling? (dates/months):",
            "5. Your main interests? (History/Beach/Adventure/Food):",
            "6. Places you've visited in Egypt before:",
            "7. Accommodation preference? (Hotel/Hostel/Airbnb):",
            "8. Transportation preference?:",
            "9. Dietary restrictions?:",
            "10. Activity level? (Few/Moderate/Full):",
            "11. Early riser or night explorer?:",
            "12. Comfort level? (Economy/Mid-range/Luxury):"
        ]

        answers = session.get("answers", {})
        current_question = len(answers) - (1 if 'selected_city' in answers else 0)

        if current_question < len(profile_fields):
            field = profile_fields[current_question]
            answers[field] = user_input
            current_question += 1

        if current_question < len(profile_questions):
            session["answers"] = answers
            session["awaiting_profile"] = True
            return jsonify({
                "type": "profile_question",
                "question": profile_questions[current_question],
                "session": session
            })

        if not answers.get("selected_city"):
            recommendations = get_top_city_recommendations(answers)
            session["recommendations"] = recommendations
            session["awaiting_city_choice"] = True
            session["awaiting_profile"] = False
            session["answers"] = answers
            cities = [
                f"{i+1}. {city} (Match Score: {score}%) – {reason}"
                for i, (city, score, reason) in enumerate(recommendations)
            ]
            return jsonify({
                "type": "city_recommendation",
                "cities": cities,
                "question": "👉 Which city would you like to plan your trip for? (Enter 1-4):",
                "session": session
            })

        try:
            plan = run_trip_planner_for_city(answers, answers["selected_city"])
            session.clear()
            return jsonify({"type": "trip_plan", "plan": plan, "session": {}})
        except Exception as e:
            return jsonify({
                "type": "error",
                "response": f"❌ Could not generate plan: {str(e)}",
                "session": session
            }), 500

    # ✅ City choice stage
    if session.get("awaiting_city_choice"):
        try:
            index = int(user_input.strip()) - 1
            recs = session.get("recommendations", [])
            if 0 <= index < len(recs):
                selected_city = recs[index][0]
                answers = session.get("answers", {})
                answers["selected_city"] = selected_city

                plan = run_trip_planner_for_city(answers, selected_city)

                session.clear()
                return jsonify({"type": "trip_plan", "plan": plan, "session": {}})
            else:
                return jsonify({
                    "type": "error",
                    "response": "❌ Please enter a valid number from 1 to 4.",
                    "session": session
                })
        except Exception as e:
            return jsonify({
                "type": "error",
                "response": f"❌ Invalid input: {str(e)}",
                "session": session
            })

    # ✅ Handle intent classification
    intent, detected_city = classify_user_intent(user_input)
    print(f"[DEBUG] Intent: {intent}, City: {detected_city}")

    if intent == "general_question":
        try:
            response = wikipedia_chain.invoke(user_input)
            return jsonify({"type": "general", "response": response, "session": session})
        except Exception as e:
            return jsonify({
                "type": "error",
                "response": f"❌ Wikipedia query failed: {str(e)}",
                "session": session
            }), 500

    elif intent == "trip_planning":
        session["awaiting_profile"] = True
        session["answers"] = {"selected_city": detected_city}
        session["city"] = detected_city
        return jsonify({
            "type": "profile_question",
            "question": "1. What type of traveler are you? (Solo/Family/Friends):",
            "session": session
        })

    elif intent == "city_specific":
        try:
            response = wikipedia_chain.invoke(user_input)
            return jsonify({"type": "general", "response": response, "session": session})
        except Exception as e:
            return jsonify({
                "type": "error",
                "response": f"❌ Wikipedia fetch failed: {str(e)}",
                "session": session
            }), 500

    else:
        return jsonify({
            "type": "unknown",
            "response": "❌ Sorry, I didn't understand that. Could you rephrase?",
            "session": session
        })

@app.route('/plan', methods=['POST'])
def plan():
    data = request.json
    user_profile = data.get('profile')
    city = data.get('city') or user_profile.get("selected_city")

    result = run_trip_planner_for_city(user_profile, city)
    return jsonify({"type": "trip_plan", "plan": result})

if __name__ == '__main__':
    from pyngrok import conf
    conf.get_default().auth_token = "2zDbxejJWEGliDfzF95RwYqve0T_3JDYKuHowy6Rs1tApgGzY"  # 🔐 Set your ngrok token here

    public_url = ngrok.connect(5000)
    print(f"✅ Public URL: {public_url}")
    app.run(port=5000)



/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:918: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


Loading travel data files...
✅ Loaded hotels: 947 records
✅ Loaded attractions: 678 records
✅ Loaded events: 52 records
✅ Loaded restaurants: 736 records
✅ Loaded transportation: 702 records
✅ Loaded flights: 53 records
✅ Loaded medical_tourism: 1291 records
✅ Public URL: NgrokTunnel: "https://ebf9-34-83-91-255.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/Jun/2025 07:38:19] "POST /chat HTTP/1.1" 200 -


🔥 Incoming session: {}
🔥 User message: I need plan to alexandria
[DEBUG] Intent: trip_planning, City: Alexandria


INFO:werkzeug:127.0.0.1 - - [30/Jun/2025 07:38:26] "POST /chat HTTP/1.1" 200 -


...

INFO:werkzeug:127.0.0.1 - - [30/Jun/2025 07:40:06] "POST /chat HTTP/1.1" 200 -


🔥 Incoming session: {'answers': {'accommodation': 'hotel', 'budget': '10000', 'dates': '10-10-2025 13-10-2025', 'duration': '3', 'food': 'halal', 'interests': 'beach', 'previous_places': 'no', 'selected_city': 'Alexandria', 'transportation': 'taxi', 'trip_type': 'family'}, 'awaiting_profile': True, 'city': 'Alexandria'}
🔥 User message: moderate


INFO:werkzeug:127.0.0.1 - - [30/Jun/2025 07:40:12] "POST /chat HTTP/1.1" 200 -


🔥 Incoming session: {'answers': {'accommodation': 'hotel', 'activity_level': 'moderate', 'budget': '10000', 'dates': '10-10-2025 13-10-2025', 'duration': '3', 'food': 'halal', 'interests': 'beach', 'previous_places': 'no', 'selected_city': 'Alexandria', 'transportation': 'taxi', 'trip_type': 'family'}, 'awaiting_profile': True, 'city': 'Alexandria'}
🔥 User message: night
🔥 Incoming session: {'answers': {'accommodation': 'hotel', 'activity_level': 'moderate', 'budget': '10000', 'dates': '10-10-2025 13-10-2025', 'duration': '3', 'food': 'halal', 'interests': 'beach', 'previous_places': 'no', 'selected_city': 'Alexandria', 'sleep_schedule': 'night', 'transportation': 'taxi', 'trip_type': 'family'}, 'awaiting_profile': True, 'city': 'Alexandria'}
🔥 User message: mid-range
 

I encountered an error while trying to use the tool. This was the error: analyze_city() got an unexpected keyword argument 'city'.
 Tool City Analysis Tool accepts these inputs: City Analysis Tool(tool_input: 'string')

INFO:werkzeug:127.0.0.1 - - [30/Jun/2025 07:41:25] "POST /chat HTTP/1.1" 200 -
